Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd

datasets_path = ''
datasets_path = 'C:/Users/Venik/OneDrive/Документы/Yandex_Praktikum/_git_YP/Linear_algebra_project'

In [2]:
insurance_dataset = pd.read_csv(datasets_path + '/datasets/insurance.csv')
insurance_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [3]:
insurance_dataset.sample(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2329,0,38.0,39100.0,0,0
3570,0,40.0,47300.0,1,0
3637,0,21.0,41000.0,1,0
2326,1,38.0,48400.0,1,0
2028,1,26.0,54400.0,1,0
1731,0,24.0,43200.0,2,0
1123,0,48.0,26700.0,3,1
58,1,20.0,24200.0,3,0
830,0,23.0,31500.0,1,0
1022,0,21.0,45800.0,0,0


## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество модели не изменится, но придется ее переобучить.

**Обоснование:** Алгоритм линейной регрессии предполагает создание коэффициента на путем создания обратимой матрицы и умножения ее на транспонированную матрицу признаков. В случае, если мы умножим на обратимую матрицу, получим следующую формулу:

$$
X_1 = XO
$$

где $O$ - обратимая матрица

Если подставить в формулы обучения и предсказания вместо матрицы $X$ матрицу $X_1$, получим:

*Для обучения:*

$$
w = (X_1^T X_1)^{-1} X_1^T y
$$

Преобразуем формулу:

$$
w = ((XO)^T XO)^{-1} (XO)^T y
$$

$$
w = (X^T O^T XO)^{-1} X^T O^T y
$$

$$
w = (X^T)^{-1} (O^T)^{-1} X^{-1} O^{-1} X^T O^T y
$$

*Произведение матирицы на обратную будет равно единичной матрице, а умножение на единичную матрицу дает исходную матрицу.* 

$$
w = (X^T)^{-1} X^{-1} O^{-1} X^T y
$$


*Подставим $w$ в формулу для предсказания:*

$$
a = XO (X^T)^{-1} X^{-1} O^{-1} X^T y
$$

*Еще раз сократим обратимую матрицу:*

$$
a = X (X^T)^{-1} X^{-1} X^T y
$$

$$
a = X (X^T X)^{-1} X^T y
$$

И получим ту же формулу, заменив $(X^T X)^{-1} X^T y$ на $w$:

$$
a = Xw
$$

**Следовательно умножение на обратимую матрицу не ухудшит качество модели**

## 3. Алгоритм преобразования

**Алгоритм**

1. Преобразуем заголовки.
2. Поделим датасет на признаки и целевой признак.
3. Сгенерируем случайным образом квадратную обратимую матрицу размером равным количеству признаков. Используем для этого метод библиотеки NumPy np.linalg.inv() В качестве среднего возьмем 1.
4. Умножим матрицу признаков на обратимую матрицу.
7. Сформируем датасет с данными из перемноженного масива признаков.
8. Сравним качество каждой из моделей методом bootstrap. Целевой признак в обоих случаях будет одинаковым.
9. Сделаем выводы по результатам сравнения качества.

**Обоснование**

Целевой признак никакой персональной информации не несет, соответственно, его можно выделить сразу. Остальные данные необходимо изменить, умножив на обратную матрицу, которую мы сформируем. Проверку произведем по методу bootstrap, т.к. он лучше позволит оценить качество модели.

## 4. Проверка алгоритма

In [4]:
import numpy as np

In [5]:
insurance_dataset.rename(columns={'Пол': 'feature_1', 
                                  'Возраст': 'feature_2',  
                                  'Зарплата': 'feature_3',  
                                  'Члены семьи': 'feature_4',
                                  'Страховые выплаты': 'target_feature',
                                 }, 
                         inplace=True)

In [6]:
def normalize_dataset(input_dataset):
    
    # Выделим целевой признак и заменим заголовки
    output_features = input_dataset.drop(['target_feature'], axis=1)
    output_target = input_dataset['target_feature']
       
    return output_features, output_target


In [7]:
insurance_features, insurance_target = normalize_dataset(insurance_dataset)   

insurance_features.sample(10)

,feature_1,feature_2,feature_3,feature_4
2369,1,30.0,55400.0,1
2338,1,27.0,45900.0,0
45,0,24.0,30400.0,1
283,1,37.0,45400.0,0
108,1,39.0,40100.0,2
1342,1,50.0,45900.0,5
2971,0,34.0,44000.0,1
1348,1,24.0,38700.0,2
4568,0,44.0,43700.0,2
2185,0,25.0,58000.0,0


In [8]:
# Сформируем обратимую матрицу
matrix = insurance_features.values
matrix_width = len(matrix[0])

mu, sigma = 1, 0.1 # mean and standard deviation
invertible_matrix = np.random.normal(mu, sigma, size = (matrix_width, matrix_width))

# Проверим обратимую матрицу
try:
    inverted_invertible_matrix = np.linalg.inv(invertible_matrix)
except:
    print('Полкчена необратимая матрица')

# Умножим исходную матрицу на обратимую и сформируем массив признаков для обучения модели
invereted_matrix = matrix @ invertible_matrix

columns = ['feature_1', 'feature_2', 'feature_3', 'feature_4']

inv_insurance_features = pd.DataFrame(invereted_matrix, columns=columns)

inv_insurance_features.sample(10)

,feature_1,feature_2,feature_3,feature_4
3059,35901.881695,33606.756331,34606.442321,32315.974158
1515,40217.114730,37646.842320,38765.485095,36197.990320
1536,56125.100461,52537.533881,54099.187222,50516.776899
2854,39985.338353,37428.777720,38542.356467,35991.242699
747,44618.600847,41766.480357,43008.121571,40160.376470
1915,49224.721483,46077.675695,47448.895398,44308.959294
1354,47405.150980,44374.634741,45694.509143,42669.906309
3008,38507.124462,36046.473356,37116.650115,34657.287060
3460,36551.603228,34215.331650,35232.751675,32900.331224
1484,47850.703946,44792.751464,46123.742272,43069.239583


In [9]:
# Создадим сравнительный датасет
compare = {'matrix_type' : [],
           'R2' : []
          }

df_compare_data = pd.DataFrame(compare)

In [10]:
# Проведем сравнение качества двух моделей 

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# На преобразованном массиве признаков
model = LinearRegression()

current_score = pd.Series(cross_val_score(model, 
                                          inv_insurance_features, 
                                          insurance_target, 
                                          scoring = 'r2',
                                          cv=5)).mean()

df_compare_data = df_compare_data.append(pd.DataFrame([['Обратная матрица', 
                                                        current_score]], 
                                                        columns = ['matrix_type', 
                                                                   'R2']),
                                                       ignore_index=True)

# На исходном массиве признаков
model = LinearRegression()

current_score = pd.Series(cross_val_score(model, 
                                          insurance_features, 
                                          insurance_target, 
                                          scoring = 'r2',
                                          cv=5)).mean()

df_compare_data = df_compare_data.append(pd.DataFrame([['Исходная матрица', 
                                                        current_score]], 
                                                        columns = ['matrix_type', 
                                                                 'R2']),
                                                      ignore_index=True)

df_compare_data

,matrix_type,R2
0,Обратная матрица,0.423114
1,Исходная матрица,0.423114


### <span style="color:green">Доказательство того, что качество модели не пострадает при умножении ее на обратимую матрицу подтвердилось. Показатель R2 одинаковый в обоих случаях.<span>
    
### <span style="color:red">Мы можем рекомендовать способ преобразования данных путем умножения его на обратимую матрицу, как способ сокрытия персональных данных.<span>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования